# Hackathon Dotz

### Importando Bibliotecas

In [0]:
# Importação dos pacotes para Analise

import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as sct
import statsmodels.api as sm
import re
import tensorflow as tf
import torch
import tqdm
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from tensorflow.keras.layers import Embedding
from tensorflow.keras import layers
from itertools import chain
from google.colab import files  


### Carregando Arquivos

In [131]:
#Carregar os dados para o google colab
uploaded = files.upload()

Saving data (1).csv to data (1).csv


In [0]:
# Importação dos dataset

df = pd.read_csv('Hackathon_Base_Treino_comdep.csv')
df2 = pd.read_csv( 'Hackathon_Base_Teste.csv')

# Etapa 1 

### Processamento de Dados

In [0]:
# Excluindo da descrição texto após os números, informações julgadas irrelevantes para a classificação.
df['DESC_AJUSTADA'] = df['DESCRIÇÃO PARCEIRO'].str.lower().str.replace('[0-9].+', '', regex=True)

# Excluindo da descrição ajustada as stop_words, informações julgadas irrelevantes para a classificação.
stop_words=['de','da','do','com','para','c/','kg','un','ml','pct','gfa','jg','p/','la','&','.',',','-','+']
for x in stop_words:
  df['DESC_AJUSTADA'] = df['DESC_AJUSTADA'].str.replace(x, '')

In [134]:
# Organizando as DEPARTAMENTO
samples = []
labels = []
class_names = []
prod_index = 0
departamento = df[['DEPARTAMENTO','DESC_AJUSTADA']].groupby(['DEPARTAMENTO']).agg(['count'])
departamento.columns = ['count']
departamento.sort_values('count', ascending=False, inplace=True)
departamento = list(departamento.index)
for dep in departamento:
    class_names.append(dep)
    produtos = len(df[df.DEPARTAMENTO == dep])
    print("Processing %s, %d products found" % (dep, produtos))
    produto = df[df.DEPARTAMENTO == dep]['DESC_AJUSTADA'].str.strip()
    #produto = list(produto.str.split().array)
    #produto = list(chain(*produto))
    #produto = sorted(set(produto))
    for prod in produto:
        samples.append(prod)
        labels.append(prod_index)    
    prod_index += 1

print("\nClasses:", class_names)
print("\nNumber of samples:", len(samples))
print("\nNumber of labels:", len(labels))

Processing MERCEARIA DOCE, 3614 products found
Processing MERCEARIA SALGADA, 2090 products found
Processing UTILIDADES DE COZINHA, 1670 products found
Processing BEBIDAS ALCOÓLICAS, 1366 products found
Processing CUIDADOS COM O CABELO, 1257 products found
Processing BEBIDAS NÃO ALCOÓLICAS, 698 products found
Processing CUIDADOS COM O CORPO, 658 products found
Processing HORTIFRUTI, 642 products found
Processing CALÇADO, 607 products found
Processing DESCARTÁVEIS, 598 products found
Processing FRIOS, 493 products found
Processing CAMA, MESA E BANHO, 453 products found
Processing CUIDADOS COM A CASA, 449 products found
Processing LAVANDERIA, 448 products found
Processing AÇOUGUE, 420 products found
Processing PAPELARIA, 387 products found
Processing IOGURTES, 360 products found
Processing ELETRO, 356 products found
Processing PET SHOP, 343 products found
Processing HIGIENE PESSOAL, 339 products found
Processing CONGELADOS DOCES, 293 products found
Processing DECORAÇÃO, 288 products found

### Dividindo Treinamento e Teste

In [0]:
# Shuffle the data
seed = 1337
rng = np.random.RandomState(seed)
rng.shuffle(samples)
rng = np.random.RandomState(seed)
rng.shuffle(labels)

# Extract a training & validation split
validation_split = 0.1
num_validation_samples = int(validation_split * len(samples))
train_samples = samples[:-num_validation_samples]
val_samples = samples[-num_validation_samples:]
train_labels = labels[:-num_validation_samples]
val_labels = labels[-num_validation_samples:]

### Crianção do vocabulário

In [0]:
vectorizer = TextVectorization(standardize='lower_and_strip_punctuation')
text_ds = tf.data.Dataset.from_tensor_slices(train_samples).batch(128)
vectorizer.adapt(text_ds)
voc = samples
word_index = dict(zip(voc, range(2, len(voc))))

### Carregar palavras pré-treinadas

In [137]:
palavras = sorted(set(samples))
embeddings_index = {item: np.random.random_sample(100) for item in tqdm.notebook.tqdm(palavras)}

print("Found %s word vectors." % len(embeddings_index))


Found 19670 word vectors.


In [138]:
num_tokens = len(voc) + 2
embedding_dim = 100
hits = 0
misses = 0

# Prepare embedding matrix
embedding_matrix = np.zeros((num_tokens, embedding_dim))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # Words not found in embedding index will be all-zeros.
        # This includes the representation for "padding" and "OOV"
        embedding_matrix[i] = embedding_vector
        hits += 1
    else:
        misses += 1
print("Converted %d words (%d misses)" % (hits, misses))

Converted 19668 words (0 misses)


In [0]:
embedding_layer = Embedding(
    num_tokens,
    embedding_dim,
    embeddings_initializer=keras.initializers.Constant(embedding_matrix),
    trainable=False,
)

### Construindo o Modelo

In [140]:
int_sequences_input = keras.Input(shape=(None,), dtype="int64")
embedded_sequences = embedding_layer(int_sequences_input)
x = layers.Conv1D(128, 1, activation="relu")(embedded_sequences)
x = layers.MaxPooling1D(1)(x)
x = layers.Conv1D(128, 1, activation="relu")(x)
x = layers.MaxPooling1D(1)(x)
x = layers.Conv1D(128, 1, activation="relu")(x)
x = layers.GlobalMaxPooling1D()(x)
x = layers.Dense(128, activation="relu")(x)
x = layers.Dropout(0.2)(x)
preds = layers.Dense(len(class_names), activation="softmax")(x)
model = keras.Model(int_sequences_input, preds)
model.summary()

Model: "model_24"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_25 (InputLayer)        [(None, None)]            0         
_________________________________________________________________
embedding_5 (Embedding)      (None, None, 100)         2201100   
_________________________________________________________________
conv1d_15 (Conv1D)           (None, None, 128)         12928     
_________________________________________________________________
max_pooling1d_10 (MaxPooling (None, None, 128)         0         
_________________________________________________________________
conv1d_16 (Conv1D)           (None, None, 128)         16512     
_________________________________________________________________
max_pooling1d_11 (MaxPooling (None, None, 128)         0         
_________________________________________________________________
conv1d_17 (Conv1D)           (None, None, 128)         165

### Treinando o Modelo

In [0]:
x_train = vectorizer(np.array([[s] for s in train_samples])).numpy()
x_val = vectorizer(np.array([[s] for s in val_samples])).numpy()

y_train = np.array(train_labels)
y_val = np.array(val_labels)

In [142]:
batch_size = 128
epochs = 70
model.compile(loss="sparse_categorical_crossentropy", optimizer="rmsprop", metrics=["accuracy"])
model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(x_val, y_val))

Epoch 1/120
155/155 [==============================] - 1s 9ms/step - loss: 3.1194 - accuracy: 0.2351 - val_loss: 2.7579 - val_accuracy: 0.3173
Epoch 2/120
155/155 [==============================] - 1s 8ms/step - loss: 2.5614 - accuracy: 0.3756 - val_loss: 2.4146 - val_accuracy: 0.3714
Epoch 3/120
155/155 [==============================] - 1s 8ms/step - loss: 2.1711 - accuracy: 0.4768 - val_loss: 2.0775 - val_accuracy: 0.4982
Epoch 4/120
155/155 [==============================] - 1s 8ms/step - loss: 1.8547 - accuracy: 0.5537 - val_loss: 1.7450 - val_accuracy: 0.5723
Epoch 5/120
155/155 [==============================] - 1s 8ms/step - loss: 1.6134 - accuracy: 0.6097 - val_loss: 1.4915 - val_accuracy: 0.6459
Epoch 6/120
155/155 [==============================] - 1s 8ms/step - loss: 1.4318 - accuracy: 0.6480 - val_loss: 1.4256 - val_accuracy: 0.6573
Epoch 7/120
155/155 [==============================] - 1s 8ms/step - loss: 1.2817 - accuracy: 0.6814 - val_loss: 1.2524 - val_accuracy: 0.6886

In [0]:
# Excluindo da descrição texto após os números, informações julgadas irrelevantes para a classificação.
df2['DESC_AJUSTADA'] = df2['DESCRIÇÃO PARCEIRO'].str.lower().str.replace('[0-9].+', '', regex=True)
# Excluindo da descrição ajustada as stop_words, informações julgadas irrelevantes para a classificação.
stop_words=['de','da','do','com','para','c/','kg','un','ml','pct','gfa','jg','p/','la','&','.',',','-','+']
for x in stop_words:
  df2['DESC_AJUSTADA'] = df2['DESC_AJUSTADA'].str.replace(x, '')

In [144]:
string_input = keras.Input(shape=(1,), dtype="string")
x = vectorizer(string_input)
preds = model(x)
end_to_end_model = keras.Model(string_input, preds)

exemplos = [[x] for x in df2['DESC_AJUSTADA']]
#exemplos = [['Exemplo']] #Informa DF
dep = []
for exemplo in tqdm.notebook.tqdm(exemplos):
  probabilities = end_to_end_model.predict([[exemplo]])
  dep.append(class_names[np.argmax(probabilities[0])])
  #print('DESCRIÇÃO: ', exemplo, '| DEPARTAMENTO: ', class_names[np.argmax(probabilities[0])])

In [145]:
# Ajustando dados de testes com as previsões
df2['DEPARTAMENTO'] = dep
df2.head(2)

,0,DESCRIÇÃO PARCEIRO,DESC_AJUSTADA,DEPARTAMENTO
0,1,SAL ROSA HIMALAIA C/ ALHO 500G,sal rosa himaia alho,MERCEARIA SALGADA
1,2,JG BOLA NATAL ACASA C/17 DR/PR/BC R952,bo natal acasa,DECORAÇÃO


# Etapa 2

In [0]:
# Excluindo da descrição texto após os números, informações julgadas irrelevantes para a classificação.
df['DESC_AJUSTADA'] = df['DESCRIÇÃO PARCEIRO'].str.lower().str.replace('[0-9].+', '', regex=True)

# Excluindo da descrição ajustada as stop_words, informações julgadas irrelevantes para a classificação.
stop_words=['de','da','do','com','para','c/','kg','un','ml','pct','gfa','jg','p/','la','&','.',',','-','+']
for x in stop_words:
  df['DESC_AJUSTADA'] = df['DESC_AJUSTADA'].str.replace(x, '')

# Incluindo o DEPARTAMENTO para Otimizar a performance
df['DESC_AJUSTADA'] = df['DEPARTAMENTO'].str.lower() +' '+ df['DESC_AJUSTADA']

In [147]:
# Organizando as CATEGORIA
samples = []
labels = []
class_names = []
prod_index = 0
categoria = df[['CATEGORIA','DESC_AJUSTADA']].groupby(['CATEGORIA']).agg(['count'])
categoria.columns = ['count']
categoria.sort_values('count', ascending=False, inplace=True)
categoria = list(categoria.index)
for cat in categoria:
    class_names.append(cat)
    produtos = len(df[df.CATEGORIA == cat])
    print("Processing %s, %d products found" % (cat, produtos))
    produto = df[df.CATEGORIA == cat]['DESC_AJUSTADA'].str.strip()
    #produto = list(produto.str.split().array)
    #produto = list(chain(*produto))
    #produto = sorted(set(produto))
    for prod in produto:
        samples.append(prod)
        labels.append(prod_index)    
    prod_index += 1

print("\nClasses:", class_names)
print("\nNumber of samples:", len(samples))
print("\nNumber of labels:", len(labels))

Processing VINHO, 789 products found
Processing BOMBONIERE, 672 products found
Processing BISCOITO, 575 products found
Processing UTENSÍLIOS DE INOX/ALUMÍNIO, 525 products found
Processing CALÇADO ADULTO, 510 products found
Processing UTENSÍLIOS DE PLÁSTICO, 446 products found
Processing QUEIJO, 413 products found
Processing CONDIMENTO NATURAL, 387 products found
Processing ARTIGOS PARA FESTAS, 378 products found
Processing FARINÁCEO, 341 products found
Processing UTENSÍLIOS DE VIDRO, 323 products found
Processing IOGURTE, 318 products found
Processing SALGADINHO E APERITIVO, 317 products found
Processing CERVEJA, 290 products found
Processing CAMA, 286 products found
Processing MATERIAL ESCOLAR, 282 products found
Processing MASSA SECA, 268 products found
Processing SUCO PRONTO/NECTAR, 240 products found
Processing PÃES, 227 products found
Processing COLORAÇÃO FEMININA, 227 products found
Processing LEGUMES, 225 products found
Processing SHAMPOO, 223 products found
Processing SORVETE,

In [0]:
# Shuffle the data
seed = 1337
rng = np.random.RandomState(seed)
rng.shuffle(samples)
rng = np.random.RandomState(seed)
rng.shuffle(labels)

# Extract a training & validation split
validation_split = 0.1
num_validation_samples = int(validation_split * len(samples))
train_samples = samples[:-num_validation_samples]
val_samples = samples[-num_validation_samples:]
train_labels = labels[:-num_validation_samples]
val_labels = labels[-num_validation_samples:]

In [0]:
vectorizer = TextVectorization(standardize='lower_and_strip_punctuation')
text_ds = tf.data.Dataset.from_tensor_slices(train_samples).batch(128)
vectorizer.adapt(text_ds)

voc = samples
word_index = dict(zip(voc, range(2, len(voc))))

In [150]:
palavras = sorted(set(samples))
embeddings_index = {item: np.random.random_sample(100) for item in tqdm.notebook.tqdm(palavras)}

print("Found %s word vectors." % len(embeddings_index))


Found 19684 word vectors.


In [151]:
num_tokens = len(voc) + 2
embedding_dim = 100
hits = 0
misses = 0

# Prepare embedding matrix
embedding_matrix = np.zeros((num_tokens, embedding_dim))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # Words not found in embedding index will be all-zeros.
        # This includes the representation for "padding" and "OOV"
        embedding_matrix[i] = embedding_vector
        hits += 1
    else:
        misses += 1
print("Converted %d words (%d misses)" % (hits, misses))

Converted 19683 words (0 misses)


In [0]:
embedding_layer = Embedding(
    num_tokens,
    embedding_dim,
    embeddings_initializer=keras.initializers.Constant(embedding_matrix),
    trainable=False,
)

In [153]:
int_sequences_input = keras.Input(shape=(None,), dtype="int64")
embedded_sequences = embedding_layer(int_sequences_input)
x = layers.Conv1D(128, 1, activation="relu")(embedded_sequences)
x = layers.MaxPooling1D(1)(x)
x = layers.Conv1D(128, 1, activation="relu")(x)
x = layers.MaxPooling1D(1)(x)
x = layers.Conv1D(128, 1, activation="relu")(x)
x = layers.GlobalMaxPooling1D()(x)
x = layers.Dense(128, activation="relu")(x)
x = layers.Dropout(0.2)(x)
preds = layers.Dense(len(class_names), activation="softmax")(x)
model = keras.Model(int_sequences_input, preds)
model.summary()

Model: "model_26"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_27 (InputLayer)        [(None, None)]            0         
_________________________________________________________________
embedding_6 (Embedding)      (None, None, 100)         2201100   
_________________________________________________________________
conv1d_18 (Conv1D)           (None, None, 128)         12928     
_________________________________________________________________
max_pooling1d_12 (MaxPooling (None, None, 128)         0         
_________________________________________________________________
conv1d_19 (Conv1D)           (None, None, 128)         16512     
_________________________________________________________________
max_pooling1d_13 (MaxPooling (None, None, 128)         0         
_________________________________________________________________
conv1d_20 (Conv1D)           (None, None, 128)         165

In [0]:
x_train = vectorizer(np.array([[s] for s in train_samples])).numpy()
x_val = vectorizer(np.array([[s] for s in val_samples])).numpy()

y_train = np.array(train_labels)
y_val = np.array(val_labels)

In [155]:
batch_size = 128
epochs = 70
model.compile(loss="sparse_categorical_crossentropy", optimizer="rmsprop", metrics=["accuracy"])
model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(x_val, y_val))

Epoch 1/120
155/155 [==============================] - 2s 10ms/step - loss: 4.1651 - accuracy: 0.1996 - val_loss: 3.0550 - val_accuracy: 0.3441
Epoch 2/120
155/155 [==============================] - 1s 8ms/step - loss: 2.5909 - accuracy: 0.3870 - val_loss: 2.1314 - val_accuracy: 0.4605
Epoch 3/120
155/155 [==============================] - 1s 8ms/step - loss: 1.9866 - accuracy: 0.4781 - val_loss: 1.7653 - val_accuracy: 0.5250
Epoch 4/120
155/155 [==============================] - 1s 8ms/step - loss: 1.6019 - accuracy: 0.5578 - val_loss: 1.4658 - val_accuracy: 0.5982
Epoch 5/120
155/155 [==============================] - 1s 8ms/step - loss: 1.3516 - accuracy: 0.6167 - val_loss: 1.2511 - val_accuracy: 0.6595
Epoch 6/120
155/155 [==============================] - 1s 8ms/step - loss: 1.1634 - accuracy: 0.6626 - val_loss: 1.1274 - val_accuracy: 0.6918
Epoch 7/120
155/155 [==============================] - 1s 8ms/step - loss: 1.0348 - accuracy: 0.6980 - val_loss: 1.0400 - val_accuracy: 0.712

In [0]:
# Incluindo o DEPARTAMENTO para Otimizar a performance
df2['DESC_AJUSTADA'] = df2['DEPARTAMENTO'].str.lower() +' '+ df['DESC_AJUSTADA']

In [157]:
string_input = keras.Input(shape=(1,), dtype="string")
x = vectorizer(string_input)
preds = model(x)
end_to_end_model = keras.Model(string_input, preds)

exemplos = [[x] for x in df2['DESC_AJUSTADA']]
#exemplos = [['Exemplo']] #Informa DF
cat = []
for exemplo in tqdm.notebook.tqdm(exemplos):
  probabilities = end_to_end_model.predict([[exemplo]])
  cat.append(class_names[np.argmax(probabilities[0])])
  #print('DESCRIÇÃO: ', exemplo, '| CATEGORIA: ', class_names[np.argmax(probabilities[0])])

In [158]:
# Ajustando dados de testes com as previsões
df2['CATEGORIA'] = cat
df2.head(2)

,0,DESCRIÇÃO PARCEIRO,DESC_AJUSTADA,DEPARTAMENTO,CATEGORIA
0,1,SAL ROSA HIMALAIA C/ ALHO 500G,mercearia salgada mercearia doce pasta int vit...,MERCEARIA SALGADA,CREME DE AMENDOIM
1,2,JG BOLA NATAL ACASA C/17 DR/PR/BC R952,decoração cuidados com a cozinha esponja betta...,DECORAÇÃO,ESPONJA SINTÉTICA


# Etapa 3

In [0]:
# Ajustando nome da coluna, para facilitar o código por causa do -
df.rename(columns={'SUB-CATEGORIA': 'SUBCATEGORIA'}, inplace=True)

In [0]:
# Excluindo da descrição texto após os números, informações julgadas irrelevantes para a classificação.
df['DESC_AJUSTADA'] = df['DESCRIÇÃO PARCEIRO'].str.lower()

# Excluindo da descrição ajustada as stop_words, informações julgadas irrelevantes para a classificação.
stop_words=['de','da','do','com','para','c/','kg','un','ml','pct','gfa','jg','p/','la','&','.',',','-','+']
for x in stop_words:
  df['DESC_AJUSTADA'] = df['DESC_AJUSTADA'].str.replace(x, '')

# Incluindo o DEPARTAMENTO e CATEGORIA para Otimizar a performance
df['DESC_AJUSTADA'] = df['DEPARTAMENTO'].str.lower() +' '+ df['CATEGORIA'].str.lower() +' '+ df['DESC_AJUSTADA']

In [161]:
# Organizando as SUB-CATEGORIA
samples = []
labels = []
class_names = []
prod_index = 0
subcategoria = df[['SUBCATEGORIA','DESC_AJUSTADA']].groupby(['SUBCATEGORIA']).agg(['count'])
subcategoria.columns = ['count']
subcategoria.sort_values('count', ascending=False, inplace=True)
subcategoria = list(subcategoria.index)
for sub in subcategoria:
    class_names.append(sub)
    produtos = len(df[df.SUBCATEGORIA == sub])
    print("Processing %s, %d products found" % (sub, produtos))
    produto = df[df.SUBCATEGORIA == sub]['DESC_AJUSTADA'].str.strip()
    #produto = list(produto.str.split().array)
    #produto = list(chain(*produto))
    #produto = sorted(set(produto))
    for prod in produto:
        samples.append(prod)
        labels.append(prod_index)    
    prod_index += 1

print("\nClasses:", class_names)
print("\nNumber of samples:", len(samples))
print("\nNumber of labels:", len(labels))

Processing TRADICIONAL, 810 products found
Processing SANDÁLIA, 572 products found
Processing REGULAR, 278 products found
Processing ESMALTE, 188 products found
Processing POTES, 183 products found
Processing VELA, 176 products found
Processing BÁSICO, 175 products found
Processing COPO, 169 products found
Processing PRATO, 165 products found
Processing HIDRATAÇÃO, 164 products found
Processing INTEGRAL, 156 products found
Processing VINHO TINTO, 153 products found
Processing VINHO CHILENO, 143 products found
Processing CADERNO, 139 products found
Processing OVO DE PÁSCOA, 134 products found
Processing MISTURAS, 123 products found
Processing AEROSOL COM PERFUME, 122 products found
Processing OUTROS, 118 products found
Processing POTE TRADICIONAL, 117 products found
Processing RECHEADO, 107 products found
Processing SECO, 105 products found
Processing ESPECIAIS, 100 products found
Processing JOGO DE CAMA, 97 products found
Processing ESPUMANTE, 95 products found
Processing PERFUMADO, 91

In [0]:
# Shuffle the data
seed = 1337
rng = np.random.RandomState(seed)
rng.shuffle(samples)
rng = np.random.RandomState(seed)
rng.shuffle(labels)

# Extract a training & validation split
validation_split = 0.1
num_validation_samples = int(validation_split * len(samples))
train_samples = samples[:-num_validation_samples]
val_samples = samples[-num_validation_samples:]
train_labels = labels[:-num_validation_samples]
val_labels = labels[-num_validation_samples:]

In [0]:
vectorizer = TextVectorization(standardize='lower_and_strip_punctuation')
text_ds = tf.data.Dataset.from_tensor_slices(train_samples).batch(128)
vectorizer.adapt(text_ds)

voc = samples
word_index = dict(zip(voc, range(2, len(voc))))

In [164]:

palavras = sorted(set(samples))
embeddings_index = {item: np.random.random_sample(100) for item in tqdm.notebook.tqdm(palavras)}

print("Found %s word vectors." % len(embeddings_index))


Found 22009 word vectors.


In [165]:
num_tokens = len(voc) + 2
embedding_dim = 100
hits = 0
misses = 0

# Prepare embedding matrix
embedding_matrix = np.zeros((num_tokens, embedding_dim))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # Words not found in embedding index will be all-zeros.
        # This includes the representation for "padding" and "OOV"
        embedding_matrix[i] = embedding_vector
        hits += 1
    else:
        misses += 1
print("Converted %d words (%d misses)" % (hits, misses))

Converted 22007 words (0 misses)


In [0]:
embedding_layer = Embedding(
    num_tokens,
    embedding_dim,
    embeddings_initializer=keras.initializers.Constant(embedding_matrix),
    trainable=False,
)

In [167]:
int_sequences_input = keras.Input(shape=(None,), dtype="int64")
embedded_sequences = embedding_layer(int_sequences_input)
x = layers.Conv1D(128, 1, activation="relu")(embedded_sequences)
x = layers.MaxPooling1D(1)(x)
x = layers.Conv1D(128, 1, activation="relu")(x)
x = layers.MaxPooling1D(1)(x)
x = layers.Conv1D(128, 1, activation="relu")(x)
x = layers.GlobalMaxPooling1D()(x)
x = layers.Dense(128, activation="relu")(x)
x = layers.Dropout(0.2)(x)
preds = layers.Dense(len(class_names), activation="softmax")(x)
model = keras.Model(int_sequences_input, preds)
model.summary()


Model: "model_28"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_29 (InputLayer)        [(None, None)]            0         
_________________________________________________________________
embedding_7 (Embedding)      (None, None, 100)         2201100   
_________________________________________________________________
conv1d_21 (Conv1D)           (None, None, 128)         12928     
_________________________________________________________________
max_pooling1d_14 (MaxPooling (None, None, 128)         0         
_________________________________________________________________
conv1d_22 (Conv1D)           (None, None, 128)         16512     
_________________________________________________________________
max_pooling1d_15 (MaxPooling (None, None, 128)         0         
_________________________________________________________________
conv1d_23 (Conv1D)           (None, None, 128)         165

In [0]:
x_train = vectorizer(np.array([[s] for s in train_samples])).numpy()
x_val = vectorizer(np.array([[s] for s in val_samples])).numpy()

y_train = np.array(train_labels)
y_val = np.array(val_labels)

In [169]:
batch_size = 128
epochs = 120
model.compile(loss="sparse_categorical_crossentropy", optimizer="rmsprop", metrics=["accuracy"])
model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(x_val, y_val))

Epoch 1/120
155/155 [==============================] - 2s 11ms/step - loss: 6.6160 - accuracy: 0.0577 - val_loss: 6.0622 - val_accuracy: 0.0918
Epoch 2/120
155/155 [==============================] - 1s 9ms/step - loss: 5.3587 - accuracy: 0.1449 - val_loss: 4.8356 - val_accuracy: 0.1868
Epoch 3/120
155/155 [==============================] - 1s 8ms/step - loss: 4.3859 - accuracy: 0.2262 - val_loss: 4.2140 - val_accuracy: 0.2809
Epoch 4/120
155/155 [==============================] - 1s 8ms/step - loss: 3.7487 - accuracy: 0.3029 - val_loss: 3.7737 - val_accuracy: 0.3391
Epoch 5/120
155/155 [==============================] - 1s 8ms/step - loss: 3.2505 - accuracy: 0.3638 - val_loss: 3.3972 - val_accuracy: 0.3918
Epoch 6/120
155/155 [==============================] - 1s 8ms/step - loss: 2.8583 - accuracy: 0.4121 - val_loss: 3.1243 - val_accuracy: 0.4286
Epoch 7/120
155/155 [==============================] - 1s 8ms/step - loss: 2.5278 - accuracy: 0.4590 - val_loss: 2.8111 - val_accuracy: 0.464

In [0]:
# Ajustando descrição.
df2['DESC_AJUSTADA'] = df2['DESCRIÇÃO PARCEIRO'].str.lower()
# Excluindo da descrição ajustada as stop_words, informações julgadas irrelevantes para a classificação.
stop_words=['de','da','do','com','para','c/','kg','un','ml','pct','gfa','jg','p/','la','&','.',',','-','+']
for x in stop_words:
  df2['DESC_AJUSTADA'] = df2['DESC_AJUSTADA'].str.replace(x, '')

# Incluindo o DEPARTAMENTO e CATEGORIA para Otimizar a performance
df2['DESC_AJUSTADA'] = df2['DEPARTAMENTO'].str.lower() +' '+ df2['CATEGORIA'].str.lower() +' '+ df2['DESC_AJUSTADA']

In [171]:
string_input = keras.Input(shape=(1,), dtype="string")
x = vectorizer(string_input)
preds = model(x)
end_to_end_model = keras.Model(string_input, preds)

exemplos = [[x] for x in df2['DESC_AJUSTADA']]
#exemplos = [['Exemplo']] #Informa DF
sub = []
for exemplo in tqdm.notebook.tqdm(exemplos):
  probabilities = end_to_end_model.predict([[exemplo]])
  sub.append(class_names[np.argmax(probabilities[0])])
  #print('DESCRIÇÃO: ', exemplo, '| DEPARTAMENTO: ', class_names[np.argmax(probabilities[0])])

In [172]:
# Ajustando dados de testes com as previsões
df2['SUB-CATEGORIA'] = sub
df2.head(2)

,0,DESCRIÇÃO PARCEIRO,DESC_AJUSTADA,DEPARTAMENTO,CATEGORIA,SUB-CATEGORIA
0,1,SAL ROSA HIMALAIA C/ ALHO 500G,mercearia salgada creme de amendoim sal rosa h...,MERCEARIA SALGADA,CREME DE AMENDOIM,TRADICIONAL
1,2,JG BOLA NATAL ACASA C/17 DR/PR/BC R952,decoração esponja sintética bo natal acasa 17...,DECORAÇÃO,ESPONJA SINTÉTICA,GANCHO


In [0]:
dotz = df2[['DESCRIÇÃO PARCEIRO', 'SUB-CATEGORIA', 'CATEGORIA']]
dotz.to_csv('dotz.csv')